# Combine the performance data from individual models

Loop over models and extract focal performance to a single table.

In [ ]:
import joblib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Loop and get the data.

In [ ]:
perf = []

for reduction in ["pca", "fa", "umap_dim5_nb20_euclidean"]:
    for model_type in ["lr", "rf"]:
        for cluster in [1, 3, 4, 5, 6, 7, 8]:
            with open(
                f"/data/uscuni-restricted/06_models/{reduction}/label_{cluster}/{model_type}/global_model.joblib",
                "rb",
            ) as f:
                model = joblib.load(f)
                perf.append(
                    pd.Series(
                        {
                            "reduction": reduction,
                            "model": model_type,
                            "cluster": cluster,
                            "accuracy": model.score_,
                            "balanced_accuracy": model.balanced_accuracy_ * 100,
                            "precision": model.precision_,
                            "recall": model.recall_,
                            "f1_macro": model.f1_macro_,
                            "f1_micro_": model.f1_micro_,
                            "f1_weighted": model.f1_weighted_,
                        }
                    )
                )

Combine.

In [ ]:
performance = pd.DataFrame(perf)

In [ ]:
performance

In [ ]:
performance.to_parquet("/data/uscuni-restricted/06_models/model_performance.parquet")

In [ ]:
performance = performance.replace(
    {
        "pca": "PCA",
        "fa": "Factor Analysis",
        "umap_dim5_nb20_euclidean": "UMAP",
        "lr": "Logistic Regression",
        "rf": "Random Forest",
    }
)

In [ ]:
ax = sns.scatterplot(
    data=performance, x="cluster", y="balanced_accuracy", hue="model", style="reduction"
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.xlabel("Cluster label")
plt.ylabel("Balanced Accuracy (%)")
plt.title("Performace of geographically weighted models")
sns.despine()

In [ ]:
ax = sns.scatterplot(
    data=performance, x="cluster", y="f1_macro", hue="model", style="reduction"
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.xlabel("Cluster label")
plt.ylabel("Balanced Accuracy (%)")
plt.title("Performace of geographically weighted models")
sns.despine()

In [ ]:
cluster_names = {
    "1": "Central Urban Developments",
    "2": "Large Scale Outliers",
    "3": "Dense Urban Developments",
    "4": "Street-aligned Developments",
    "5": "Sparse Rural Development",
    "6": "Linear Road Network Developments",
    "7": "Sparse Road Network Developments",
    "8": "Large Scale Developments",
}